# TRANSPORT PTOPERTIES OF CUPRATES - FLEX-CVC 

In this study, we investigate the transport phenomena in high-$T_c$ superconductors (HTSCs) based on the Fermi liquid theory, and perform a numerical study using the spin fluctuation theory. We analyze the following Hubbard model:

$$ H = \sum_{\boldsymbol{k}σ} \varepsilon_{\boldsymbol{k}}^0 c^\dagger_{\boldsymbol{k}σ} c_{\boldsymbol{k}σ} + U \sum_{\boldsymbol{k}\boldsymbol{k}'\boldsymbol{q}} c^\dagger_{\boldsymbol{k}+\boldsymbol{q}\uparrow} c^\dagger_{\boldsymbol{k}'-\boldsymbol{q}\downarrow} c_{\boldsymbol{k}'\downarrow} c_{\boldsymbol{k}\uparrow} \tag{1}$$

where $U$ is the Coulomb interaction, $c^\dagger_{\boldsymbol{k}σ}$ and $c_{\boldsymbol{k}σ}$ are the creation and annihilation operators of an electron with momentum $\boldsymbol{k}$ and spin $\sigma$ and $\varepsilon_{\boldsymbol{k}}^0$ is the spectrum of the conduction electron, that we will model with a 2D tight-binding dispersion.

<a id='eq-tb'></a>
$$\begin{align}
\varepsilon^0_{\boldsymbol{k}} = \mu + \frac{t_1}{2}\left[\cos(k_xa)+\cos(k_yb)\right] &+ t_2\cos(k_xa)\cos(k_yb)+\frac{t_3}{2}\left[\cos(2k_xa)+\cos(2k_yb)\right] + \tag{2} \\ 
&+ \frac{t_4}{2}\left[\cos(2k_xa)\cos(k_yb)+\cos(k_xa)\cos(2k_yb)\right] + t_5\cos(2k_xa)\cos(2k_yb)
\end{align}$$

where $\mu$ is the chemical potential, $t_1$, $t_2$, $t_3$, $t_4$ and $t_5$ are the hopping parameters up to fifth nearest-neighbour interactions and $a$, $b$ and $c$ are the lattice parameters.

Here, we discuss the functional form of the dynamical spin susceptibility $\chi^{s}_{\boldsymbol{q}}(\omega)$ in nearly antiferromagnetic (AF) metals. Hereafter, we use the unit $c = \hbar = k_B = 1$. This is the most
important physical quantity in such metals since it is the origin of various non-Fermi liquid behaviors in HTSCs. We study the self-energy, dynamical spin susceptibility and the CVC using the FLEX approximation[$^{[1]}$](#ref-Bic). The FLEX approximation is classified as a conserving approximation whose framework was constructed by Baym and Kadanoff[$^{[2]}$](#ref-Bay1) and by Baym[$^{[3]}$](#ref-Bay2). For this reason, we can calculate the CVC without ambiguity, by following the Ward identity $\Gamma^I = \delta\Sigma/\delta G$. Here, the Green function and the self-energy are given by

$$\begin{align}
&G_{\boldsymbol{k}}(\varepsilon_n) = (i\varepsilon_n - \varepsilon_{\boldsymbol{k}}^0 - \Sigma_{\boldsymbol{k}}(\varepsilon_n))^{-1} \tag{3} \\
&\Sigma_{\boldsymbol{k}}(\varepsilon_n) = T \sum_{\boldsymbol{q},l} G_{\boldsymbol{k}-\boldsymbol{q}}(\varepsilon_n-\omega_l) V_{\boldsymbol{q}}(\omega_l) \tag{4} \\
&V_{\boldsymbol{q}}(\omega_l) = U^2 \left(\frac{3}{2} \chi^s_{\boldsymbol{q}}(\omega_l)+\frac{1}{2} \chi^c_{\boldsymbol{q}}(\omega_l) - \chi^0_{\boldsymbol{q}}(\omega_l) \right) + U \tag{5} \\
&\chi^{s(c)}_{\boldsymbol{q}}(\omega_l) = \chi^{0}_{\boldsymbol{q}} \{1-(+)U\chi^{0}_{\boldsymbol{q}}(\omega_l) \}^{-1} \tag{6} \\
&\chi^{0}_{\boldsymbol{q}}(\omega_l) = -T\sum_{\boldsymbol{k},n} G_{\boldsymbol{k}+\boldsymbol{q}}(\varepsilon_n+\omega_l) G_{\boldsymbol{k}}(\varepsilon_n) \tag{7}
\end{align}$$

where $\varepsilon_n = (2n+1)\pi T$ and $\omega_l = 2l\pi T$, and $\mu$ refers to the chemical potential.

<a id='img-FLEX'></a>
<table>
<caption>
Fig.1 - Diagrammatic representation of the self-energy in the one-loop (FLEX) approximation<a href="#ref-Kon">$^{[4]}$</a>.
</caption>
<tbody>
<tr>
<td><img src="Figures/S-FLEX.png" alt="E" width="300"/></td>
</tr>
</tbody>
</table>

In [2]:
import numpy as np
from numba import njit, prange
from metals import *

from time import time

import matplotlib.pyplot as plt

%matplotlib qt5

In [ ]:
@njit(fastmath=True)
def get_E0(kx, ky):
    """Compute and return the in-plane dispersion."""
    
    E1 = t1/2*(np.cos(kx*a) + np.cos(ky*b)) # first nearest neighbour hopping energy
    E2 = t2*np.cos(kx*a)*np.cos(ky*b) # second n. n.
    E3 = t3/2*(np.cos(2*kx*a) + np.cos(2*ky*b)) # third n. n.
    E4 = t4/2*(np.cos(2*kx*a)*np.cos(ky*b) + np.cos(kx*a)*np.cos(2*ky*b)) # fourth n. n.
    E5 = t5*np.cos(2*kx*a)*np.cos(2*ky*b) # fifth n. n.
    
    return mu+E1+E2+E3+E4+E5

@njit(fastmath=True)
def get_GF0(kx, ky, wn):
    return 1/(1j*wn - get_E0(kx, ky))

@njit(fastmath=True)
def get_chi0(qx, qy, wl, U, T, n_cut, N_k, kx0, ky0, wn0, qx0, qy0, wl0):
    chi0 = 0
    for i1 in prange(2*n_cut+1):
        for i2 in prange(N_k):
            for i3 in prange(N_k):
                GF0_1 = get_GF0(kx0[i2]+qx, ky0[i3]+qy, wn0[i1]+wl)
                GF0_2 = get_GF0(kx0[i2], ky0[i3], wn0[i1])
                chi0 = chi0 + GF0_1*GF0_2
    return -(kB*T)*chi0

@njit(fastmath=True)
def get_V0(qx, qy, wl, U, T, n_cut, N_k, kx0, ky0, wn0, qx0, qy0, wl0):
    chi0 = get_chi0(qx, qy, wl, U, T, n_cut, N_k, kx0, ky0, wn0, qx0, qy0, wl0)
    x = U*chi0
    return U*(1 + x/2*(x**2 + 2*x + 3)/(1-x**2))

@njit(fastmath=True)
def get_selfE0(kx, ky, wn, U, T, n_cut, N_k, kx0, ky0, wn0, qx0, qy0, wl0):
    selfE0 = 0
    for i1 in prange(2*n_cut+1):
        for i2 in prange(N_k//2-1):
            for i3 in prange(N_k//2-1):
                GF0 = get_GF0(kx-qx0[i2], ky-qy0[i3], wn-wl0[i1])
                V0 = get_V0(qx0[i2], qy0[i3], wl0[i1], U, T, n_cut, N_k, kx0, ky0, wn0, qx0, qy0, wl0)
                selfE0 = selfE0 + GF0*V0
    return (kB*T)*selfE0

@njit(fastmath=True)
def get_GF1(kx, ky, wn, U, T, n_cut, N_k, kx0, ky0, wn0, qx0, qy0, wl0):
    return 1/(1j*wn - get_E0(kx, ky) - get_selfE0(kx, ky, wn, U, T, n_cut, N_k, kx0, ky0, wn0, qx0, qy0, wl0))

@njit(fastmath=True)
def get_chi1(qx, qy, wl, U, T, n_cut, N_k, kx0, ky0, wn0, qx0, qy0, wl0):
    chi1 = 0
    for i1 in prange(2*n_cut+1):
        for i2 in prange(N_k):
            for i3 in prange(N_k):
                GF1_1 = get_GF1(kx0[i2]+qx, ky0[i3]+qy, wn0[i1]+wl, U, T, n_cut, N_k, kx0, ky0, wn0, qx0, qy0, wl0)
                GF1_2 = get_GF1(kx0[i2], ky0[i3], wn0[i1], U, T, n_cut, N_k, kx0, ky0, wn0, qx0, qy0, wl0)
                chi1 = chi1 + GF1_1*GF1_2
    return -(kB*T)*chi1

@njit(fastmath=True)
def get_V1(qx, qy, wl, U, T, n_cut, N_k):
    chi1 = get_chi1(qx, qy, wl, U, T, n_cut, N_k)
    x = U*chi1
    return U*(1 + x/2*(x**2 + 2*x + 3)/(1-x**2))

@njit(fastmath=True)
def get_selfE1(kx, ky, wn, U, T, n_cut, N_k):
    selfE1 = 0
    qy = np.linspace(-np.pi/b, np.pi/b, N_k)
    for l in np.arange(-n_cut, n_cut+1):
        wl = 2*l*np.pi*(kB*T)
        for qx in np.linspace(-np.pi/a, np.pi/a, N_k):
            selfE1 += np.sum(get_GF1(kx-qx, ky-qy, wn-wl, U, T, n_cut, N_k)*get_V1(qx, qy, wl, U, T, n_cut, N_k))
    return (kB*T)*selfE1

@njit(fastmath=True)
def get_GF2(kx, ky, wn, U, T, n_cut, N_k):
    return 1/(1j*hbar*wn - get_E0(kx, ky) - get_selfE1(kx, ky, wn, U, T, n_cut, N_k))

@njit(fastmath=True)
def get_chi2(qx, qy, wl, U, T, n_cut, N_k):
    chi2 = 0
    ky = np.linspace(-np.pi/b, np.pi/b, N_k)
    for n in np.arange(-n_cut, n_cut+1):
        wn = (2*n+1)*np.pi*(kB*T)
        for kx in np.linspace(-np.pi/a, np.pi/a, N_k):
            chi2 -= np.sum(get_GF2(kx+qx, ky+qy, wn+wl, U, T, n_cut, N_k)*get_GF2(kx, ky, wn, U, T, n_cut, N_k))
    return (kB*T)*chi2

@njit(fastmath=True)
def get_V2(qx, qy, wl, U, T, n_cut, N_k):
    chi2 = get_chi2(qx, qy, wl, U, T, n_cut, N_k)
    x = U*chi2
    return U*(1 + x/2*(x**2 + 2*x + 3)/(1-x**2))

@njit(fastmath=True)
def get_selfE2(kx, ky, wn, U, T, n_cut, N_k):
    selfE2 = 0
    qy = np.linspace(-np.pi/b, np.pi/b, N_k)
    for l in np.arange(-n_cut, n_cut+1):
        wl = 2*l*np.pi*(kB*T)
        for qx in np.linspace(-np.pi/a, np.pi/a, N_k):
            selfE2 += np.sum(get_GF2(kx-qx, ky-qy, wn-wl, U, T, n_cut, N_k)*get_V2(qx, qy, wl, U, T, n_cut, N_k))
    return (kB*T)*selfE2

@njit(fastmath=True)
def get_GF3(kx, ky, wn, U, T, n_cut, N_k):
    return 1/(1j*hbar*wn - get_E0(kx, ky) - get_selfE2(kx, ky, wn, U, T, n_cut, N_k))

@njit(fastmath=True)
def get_chi3(qx, qy, wl, U, T, n_cut, N_k):
    chi3 = 0
    ky = np.linspace(-np.pi/b, np.pi/b, N_k)
    for n in np.arange(-n_cut, n_cut+1):
        wn = (2*n+1)*np.pi*(kB*T)
        for kx in np.linspace(-np.pi/a, np.pi/a, N_k):
            chi3 -= np.sum(get_GF3(kx+qx, ky+qy, wn+wl, U, T, n_cut, N_k)*get_GF3(kx, ky, wn, U, T, n_cut, N_k))
    return (kB*T)*chi3

In [ ]:
LSCO = Metal('LSCO', p=0.16)

N_k = 17
n_cut = 1
U = 6*e
T = 10

a, b, c = LSCO.s_coeff[:3] # lattice constants
mu, t1, t2, t3, t4, t5 = LSCO.s_coeff[3:9] # TB parameters

n = np.arange(-n_cut, n_cut+1)
kx0 = np.linspace(-np.pi/a, np.pi/a, N_k)
ky0 = np.linspace(-np.pi/b, np.pi/b, N_k)
wn0 = (2*n+1)*np.pi*(kB*T)

l = np.arange(-n_cut, n_cut+1)
wl0 = 2*l*np.pi*(kB*T)
qx0 = np.linspace(0, np.pi/a, N_k//2)[1:]
qy0 = np.linspace(0, np.pi/b, N_k//2)[1:]
    
qx, qy = np.linspace(0, np.pi/a, 8)[1:], np.linspace(0, np.pi/b, 8)[1:]
wl = 0

@njit(fastmath=True, parallel=True)
def get_chi(qx, qy, wl, U, T, n_cut, N_k, kx0, ky0, wn0, qx0, qy0, wl0):
    chi = np.zeros((len(qx), len(qy)), dtype=np.complex64)
    for i1 in prange(len(qx)):
        for i2 in prange(len(qy)):
            chi[i1,i2] = get_chi1(qx=qx[i1], qy=qy[i2], wl=wl, U=U, T=T, n_cut=n_cut, N_k=N_k, 
                                  kx0=kx0, ky0=ky0, wn0=wn0, qx0=qx0, qy0=qy0, wl0=wl0)
    return chi
            
t0 = time()
chi = get_chi(qx, qy, wl, U, T, n_cut, N_k, kx0, ky0, wn0, qx0, qy0, wl0)
print(time()-t0)
        
qx, qy = np.meshgrid(qx, qy)

fig, ax = plt.subplots(subplot_kw={'projection':'3d'})
ax.plot_surface(qx, qy, np.real(chi))
#ax.plot_surface(qx, qy, np.real(chi0))
#ax.plot_surface(qx, qy, np.real(chi)/np.real(chi0))

plt.show()

In [ ]:
LSCO = Metal('LSCO', p=0.16)

N_k = 2599
n_cut = 1
U = 6*e
T = 10

a, b, c = LSCO.s_coeff[:3] # lattice constants
mu, t1, t2, t3, t4, t5 = LSCO.s_coeff[3:9] # TB parameters

n = np.arange(-n_cut, n_cut+1)
kx0 = np.linspace(-np.pi/a, np.pi/a, N_k)
ky0 = np.linspace(-np.pi/b, np.pi/b, N_k)
wn0 = (2*n+1)*np.pi*(kB*T)

l = np.arange(-n_cut, n_cut+1)
wl0 = 2*l*np.pi*(kB*T)
qx0 = np.linspace(0, np.pi/a, 17)[1:]
qy0 = np.linspace(0, np.pi/b, 17)[1:]

GF0_k = np.zeros((2*n_cut+1, N_k, N_k), dtype=np.complex128)
GF0_kq = np.zeros_like(GF0_k)
Chi0 = np.zeros((16,16), dtype=np.float64)

@njit(fastmath=True)
def get_E0(kx, ky):
    """Compute and return the in-plane dispersion."""
    
    E1 = t1/2*(np.cos(kx*a) + np.cos(ky*b)) # first nearest neighbour hopping energy
    E2 = t2*np.cos(kx*a)*np.cos(ky*b) # second n. n.
    E3 = t3/2*(np.cos(2*kx*a) + np.cos(2*ky*b)) # third n. n.
    E4 = t4/2*(np.cos(2*kx*a)*np.cos(ky*b) + np.cos(kx*a)*np.cos(2*ky*b)) # fourth n. n.
    E5 = t5*np.cos(2*kx*a)*np.cos(2*ky*b) # fifth n. n.
    
    return mu+E1+E2+E3+E4+E5

@njit(parallel=True, fastmath=True)
def get_Chi0(Chi0, N_k, n_cut, GF0_k, GF0_kq, kx0, ky0, wn0, qx0, qy0):
    for i1 in range(16):
        for i2 in range(16):
            for i3 in prange(2*n_cut+1):
                for i4 in prange(N_k):
                    for i5 in prange(N_k):
                        GF0_k[i3,i4,i5] = 1/(1j*wn0[i3] - get_E0(kx0[i4], ky0[i5]))
                        GF0_kq[i3,i4,i5] = 1/(1j*wn0[i3] - get_E0(kx0[i4]+qx0[i1], ky0[i5]+qy0[i2]))

            Chi0[i1,i2] = -kB*T*np.sum(np.real(GF0_kq*GF0_k))
    return Chi0
        
Chi0 = get_Chi0(Chi0, N_k, n_cut, GF0_k, GF0_kq, kx0, ky0, wn0, qx0, qy0)
qx, qy = np.meshgrid(qx0, qy0)

fig, ax = plt.subplots(subplot_kw={'projection':'3d'})
ax.plot_surface(qx, qy, np.real(Chi0))
#ax.plot_surface(qx, qy, np.real(chi0))
#ax.plot_surface(qx, qy, np.real(chi)/np.real(chi0))

plt.show()

In [ ]:
wn0/e, LSCO.s_coeff[3:]/e

## BIBLIOGRAPHY

1. <a id='ref-Bic'></a> N. E. Bickers and S. R. White, Conserving approximations for strongly fluctuating electron systems. II. Numerical results and parquet extension, [*Phys. Rev. B* **43**, 8044 (1991)](https://doi.org/10.1103/PhysRevB.43.8044).

2. <a id='ref-Bay1'></a> G. Baym and L. P. Kadanoff, Conservation Laws and Correlation Functions, [*Phys. Rev.* **124**, 287 (1961)](https://doi.org/10.1103/PhysRev.124.287).

3. <a id='ref-Bay2'></a> G. Baym, Self-Consistent Approximations in Many-Body Systems, [*Phys. Rev.* **127**, 1391 (1962)](https://doi.org/10.1103/PhysRev.127.1391).

4. <a id='ref-Kon'></a> H. Kontani, Anomalous transport phenomena in Fermi liquids with strong magnetic fluctuations, [*Rep. Prog. Phys.* **71**, 026501 (2008)](https://doi.org/10.1088/0034-4885/71/2/026501).